## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score, accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer

## Data Prep

In [2]:
df = pd.read_csv('course_lead_scoring.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lead_source               1334 non-null   object 
 1   industry                  1328 non-null   object 
 2   number_of_courses_viewed  1462 non-null   int64  
 3   annual_income             1281 non-null   float64
 4   employment_status         1362 non-null   object 
 5   location                  1399 non-null   object 
 6   interaction_count         1462 non-null   int64  
 7   lead_score                1462 non-null   float64
 8   converted                 1462 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 102.9+ KB


In [3]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [4]:
categorical_columns = df.select_dtypes(include=object).columns.to_list()
categorical_columns

['lead_source', 'industry', 'employment_status', 'location']

In [5]:
numerical_columns = list(set(df.columns) - set(categorical_columns))
numerical_columns.remove("converted")
numerical_columns

['lead_score',
 'number_of_courses_viewed',
 'annual_income',
 'interaction_count']

In [6]:
df[numerical_columns] = df[numerical_columns].fillna(0.0)
df[categorical_columns] = df[categorical_columns].fillna("NA")

## Question 1: What is the mode of the `industry` variable?

In [7]:
df['industry'].mode()[0]

'retail'

## Question 2: What are the two features that have the biggest correlation?

Create the correlation matrix for the numerical features. Check these pairs:
- interaction_count and lead_score
- number_of_courses_viewed and lead_score
- number_of_courses_viewed and interaction_count
- annual_income and interaction_count

In [8]:
corr_matrix = df[numerical_columns].corr()
corr_matrix

,lead_score,number_of_courses_viewed,annual_income,interaction_count
lead_score,1.000000,-0.004879,0.015610,0.009888
number_of_courses_viewed,-0.004879,1.000000,0.009770,-0.023565
annual_income,0.015610,0.009770,1.000000,0.027036
interaction_count,0.009888,-0.023565,0.027036,1.000000


The pair with the highest correlation would be annual_income and interaction_count at .027036

## Question 3: Which categorical variable has the biggest mutual information score?

Calculate the mutual information score between y and other categorical variables using the training set only. Round scores to 2 decimals.

In [9]:
# Split data: 60% train, 20% val, 20% test with seed 42
# First split: 60% train, 40% temp (val + test)
df_train, df_temp = train_test_split(df, test_size=0.4, random_state=42)

# Second split: split the 40% into 50/50 (20% val, 20% test)
df_valid, df_test = train_test_split(df_temp, test_size=0.5, random_state=42)


df_train = df_train.reset_index(drop=True)
df_valid = df_valid.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [10]:
y_train = df_train["converted"].values
y_valid = df_valid["converted"].values
y_test = df_test["converted"].values

del df_train["converted"]
del df_valid["converted"]
del df_test["converted"]

In [11]:
def mutual_info_convert_score(series):
    return mutual_info_score(series, y_train)

In [12]:
mi = df_train[categorical_columns].apply(mutual_info_convert_score)
mi.sort_values(ascending=False)
mi.round(2)

lead_source          0.03
industry             0.02
employment_status    0.02
location             0.00
dtype: float64

The highest MI score is for the variable 'leadsource'

## Data Preparation for next question

In [13]:
ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
X_train_cat = ohe.fit_transform(df_train[categorical_columns])
ohe.get_feature_names_out()

array(['lead_source_NA', 'lead_source_events',
       'lead_source_organic_search', 'lead_source_paid_ads',
       'lead_source_referral', 'lead_source_social_media', 'industry_NA',
       'industry_education', 'industry_finance', 'industry_healthcare',
       'industry_manufacturing', 'industry_other', 'industry_retail',
       'industry_technology', 'employment_status_NA',
       'employment_status_employed', 'employment_status_self_employed',
       'employment_status_student', 'employment_status_unemployed',
       'location_NA', 'location_africa', 'location_asia',
       'location_australia', 'location_europe', 'location_middle_east',
       'location_north_america', 'location_south_america'], dtype=object)

## Question 4: What's the accuracy on the validation dataset?

Train a logistic regression model with one-hot encoding for categorical variables. Use these parameters for reproducibility:
- solver='liblinear'
- C=1.0
- max_iter=1000
- random_state=42

Round the accuracy to 2 decimal digits.

In [14]:
X_train_num = df_train[numerical_columns].values
X_train = np.column_stack([X_train_num, X_train_cat])
model = LogisticRegression(solver='liblinear', C=100, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,100
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [15]:
X_valid_num = df_valid[numerical_columns].values
X_valid_cat = ohe.transform(df_valid[categorical_columns])
X_valid = np.column_stack([X_valid_num, X_valid_cat])
y_pred = model.predict(X_valid)
accuracy = (y_valid == y_pred).mean()
accuracy
print(f'Accuracy: {accuracy}\nRounded accuracy: {round(accuracy, 2)}')

Accuracy: 0.7431506849315068
Rounded accuracy: 0.74


## Question 5: Which feature has the smallest difference?

Find the least useful feature using feature elimination technique:
1. Train a model with all features (same parameters as Q4, without rounding)
2. Exclude each feature one at a time and train a model without it
3. Record the accuracy for each model
4. Calculate the difference: (original accuracy - accuracy without feature)

Which of these features has the smallest difference?
- 'industry'
- 'employment_status'
- 'lead_score'

Note: The difference doesn't have to be positive.

In [16]:
# without 'industry'
dicts_train1 = df_train[['lead_source', 'employment_status', 'location']+numerical_columns].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
X_train1 = dv.fit_transform(dicts_train1)
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train1, y_train)

dicts_val1 = df_valid[['lead_source', 'employment_status', 'location']+numerical_columns].to_dict(orient='records')
X_val1 = dv.transform(dicts_val1)
y_pred1 = model.predict(X_val1)

acc1 = (y_valid == y_pred1).mean()
print(acc1)
print(accuracy - acc1)

0.7431506849315068
0.0


In [17]:
# without 'employment_status'
dicts_train1 = df_train[['lead_source', 'industry', 'location']+numerical_columns].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
X_train1 = dv.fit_transform(dicts_train1)
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train1, y_train)

dicts_val1 = df_valid[['lead_source', 'industry', 'location']+numerical_columns].to_dict(orient='records')
X_val1 = dv.transform(dicts_val1)
y_pred1 = model.predict(X_val1)

acc1 = (y_valid == y_pred1).mean()
print(acc1)
print(accuracy - acc1)

0.7465753424657534
-0.003424657534246589


In [18]:
# without 'lead_source'
dicts_train1 = df_train[['employment_status', 'industry', 'location']+numerical_columns].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
X_train1 = dv.fit_transform(dicts_train1)
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train1, y_train)

dicts_val1 = df_valid[['employment_status', 'industry', 'location']+numerical_columns].to_dict(orient='records')
X_val1 = dv.transform(dicts_val1)
y_pred1 = model.predict(X_val1)

acc1 = (y_valid == y_pred1).mean()
print(acc1)
print(accuracy - acc1)

0.7294520547945206
0.013698630136986245


## Question 6: Which C parameter leads to the best accuracy?

Train regularized logistic regression models with different C values: [0.01, 0.1, 1, 10, 100]

Use all features as in Q4 with the same parameters (except varying C).
Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

Which C leads to the best accuracy on the validation set?
- 0.01
- 0.1
- 1
- 10
- 100

Note: If there are multiple options with the same accuracy, select the smallest C.

In [24]:
# Split data: 60% train, 20% val, 20% test with seed 42
# First split: 60% train, 40% temp (val + test)
df_train, df_temp = train_test_split(df, test_size=0.4, random_state=42)

# Second split: split the 40% into 50/50 (20% val, 20% test)
df_valid, df_test = train_test_split(df_temp, test_size=0.5, random_state=42)


df_train = df_train.reset_index(drop=True)
df_valid = df_valid.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

# Prepare data with one-hot encoding
dv = DictVectorizer(sparse=False)
train_dict = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_valid.to_dict(orient='records')
X_val = dv.transform(val_dict)

# Try different C values
C_values = [0.01, 0.1, 1, 10, 100]
results = {}

print("Testing different C values:")
print("="*60)

for C in C_values:
    # Train model with current C value
    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    
    # Predict and calculate accuracy
    y_pred = model.predict(X_val)
    accuracy = accuracy_score(y_valid, y_pred)
    
    # Round to 3 decimal digits
    accuracy_rounded = round(accuracy, 3)
    results[C] = accuracy_rounded
    
    print(f"C={C:>6}: accuracy = {accuracy}, rounded = {accuracy_rounded}")

print("\n" + "="*60)

# Find the best C (highest accuracy, smallest C if tied)
max_accuracy = max(results.values())
best_C = min([c for c, acc in results.items() if acc == max_accuracy])

print(f"\nBest accuracy: {max_accuracy}")
print(f"Best C: {best_C}")

best_C

Testing different C values:
C=  0.01: accuracy = 0.863013698630137, rounded = 0.863
C=   0.1: accuracy = 0.8458904109589042, rounded = 0.846
C=     1: accuracy = 0.8424657534246576, rounded = 0.842
C=    10: accuracy = 0.8424657534246576, rounded = 0.842
C=   100: accuracy = 0.8424657534246576, rounded = 0.842


Best accuracy: 0.863
Best C: 0.01


0.01